In [ ]:
import nltk
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('treebank')

In [ ]:
nltk.corpus.treebank.tagged_sents()

In [ ]:
tagged_sentences = nltk.corpus.treebank.tagged_sents() # 토큰화에 품사 태깅이 된 데이터 받아오기
print("품사 태깅이 된 문장 개수: ", len(tagged_sentences)) # 문장 샘플의 개수 출력

In [ ]:
tagged_sentences[3913]

In [ ]:
x = [1, 2, 3, 4]
y = [4, 5, 6]
#x='abc'
#y='def' 
zipped = zip(x, y)
print(list(zipped))


In [ ]:
from itertools import zip_longest
x = [1, 2, 3]
y = [4, 5, 6,7]
 
# 1) zip_longest 적용
zipped = zip_longest(x, y)
print(list(zipped))
'''
결과
[(1, 4), (2, 5), (3, 6), (None, 7)]
'''
 
# 2) fillvalue= 인자에 값을 지정
zipped = zip_longest(x, y, fillvalue=0)
print(list(zipped))


In [ ]:
sentences, pos_tags = [], [] 
for tagged_sentence in tagged_sentences: # 3,914개의 문장 샘플을 1개씩 불러온다.
    sentence, tag_info = zip(*tagged_sentence) # 각 샘플에서 단어들은 sentence에 품사 태깅 정보들은 tag_info에 저장한다.
    sentences.append(list(sentence)) # 각 샘플에서 단어 정보만 저장한다.
    pos_tags.append(list(tag_info)) # 각 샘플에서 품사 태깅 정보만 저장한다.

In [ ]:
print(len(sentences[1]))
print(len(pos_tags[1]))

In [ ]:
len(pos_tags)

In [ ]:
#sentences 최대 길이와 평균길이를 출력
print('샘플의 최대 길이 : %d' % max(len(l) for l in sentences))
print('샘플의 평균 길이 : %f' % (sum(map(len, sentences))/len(sentences)))
plt.hist([len(s) for s in sentences], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def tokenize(samples):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(samples)
  return tokenizer

In [ ]:
src_tokenizer = tokenize(sentences)
tar_tokenizer = tokenize(pos_tags)

In [ ]:
vocab_size=len(src_tokenizer.word_index)+1 #11387+1

In [ ]:
tag_size=len(tar_tokenizer.word_index)+1 #46+1

In [ ]:
tag_size

In [ ]:
X_train = src_tokenizer.texts_to_sequences(sentences)
y_train = tar_tokenizer.texts_to_sequences(pos_tags)

In [ ]:
y_train[0]

In [ ]:
max_len = 100
X_train = pad_sequences(X_train, padding='post', maxlen=max_len)
# X_train의 모든 샘플의 길이를 맞출 때 뒤의 공간에 숫자 0으로 채움.
y_train = pad_sequences(y_train, padding='post', maxlen=max_len)
# y_train의 모든 샘플의 길이를 맞출 때 뒤의 공간에 숫자 0으로 채움.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=.2, random_state=777)


In [ ]:
y_train = to_categorical(y_train, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)

In [ ]:
print('훈련 샘플 문장의 크기 : {}'.format(X_train.shape))
print('훈련 샘플 레이블의 크기 : {}'.format(y_train.shape))
print('테스트 샘플 문장의 크기 : {}'.format(X_test.shape))
print('테스트 샘플 레이블의 크기 : {}'.format(y_test.shape))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, Embedding
from keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dense(tag_size, activation=('softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=6,  validation_data=(X_test, y_test))


In [ ]:
# mask_zero=True
# 0으로 패딩된 값을 마스킹하여 네트워크의 뒤로 전달되지 않게 함.
# 인위적으로 패딩된 부분이 학습에 영향을 미치지 않도록 하겠다는 의미임

print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))


In [ ]:
index_to_word=src_tokenizer.index_word
index_to_tag=tar_tokenizer.index_word

In [ ]:
X_test[10] #테스트 문장을 구성하는 각각 단어의 index

In [ ]:
y_test[10] #테스트 문장을 구성하는 각각 단어의 품사

In [ ]:
y_predicted = model.predict(X_test[10])

In [ ]:
y_predicted=np.argmax(y_predicted, axis=-1)  #100,1,47에서 47에 해당되는 축이 추출

In [ ]:
np.argmax(y_test[10],axis=-1)

In [ ]:
y_predicted

In [ ]:
index_to_word=src_tokenizer.index_word
index_to_tag=tar_tokenizer.index_word

i=10 # 확인하고 싶은 테스트용 샘플의 인덱스.
y_predicted = model.predict(np.array([X_test[i]])) # 입력한 테스트용 샘플에 대해서 예측 y를 리턴
y_predicted = np.argmax(y_predicted, axis=-1) # 원-핫 인코딩을 다시 정수 인코딩으로 변경함.
true = np.argmax(y_test[i], -1) # 원-핫 인코딩을 다시 정수 인코딩으로 변경함.

print("{:15}|{:5}|{}".format("단어", "실제값", "예측값"))
print(35 * "-")

for w, t, pred in zip(X_test[i], true, y_predicted[0]):
    if w != 0: # PAD값은 제외함.
        print("{:17}: {:7} {}".format(index_to_word[w], index_to_tag[t].upper(), index_to_tag[pred].upper()))

In [ ]:
#개체명 인식
# 삼성전자 - 기업
# 2020년 - 시간
# 길동 - 사람
# ...

In [ ]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
from nltk import word_tokenize, pos_tag, ne_chunk


In [ ]:
sentence = "James is working at Apple in Gangnam"

In [ ]:
#word_tokenize(sentence)
sentence = pos_tag(word_tokenize(sentence))
#NNP:고유명사

In [ ]:
print(ne_chunk(sentence))

In [ ]:
# 참고주소
# https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt
# https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html